# For expressions

After completing this lesson, you should be able to:
  - Understand the relationship between for expressions and higher order functions
  - Describe the usage of for expressions

For expressions are syntactic sugar that simplifies the work of programming a multistage transformation

Let's take an example:

In [3]:
myNums.map(i => (1 to i))

res2: scala.collection.immutable.IndexedSeq[scala.collection.immutable.Range.Inclusive] = Vector(Range 1 to 1, Range 1 to 2, Range 1 to 3)


In [2]:
val myNums = 1 to 3
myNums.map(i => (1 to i).map(j => i * j))

myNums: scala.collection.immutable.Range.Inclusive = Range 1 to 3
res1: scala.collection.immutable.IndexedSeq[scala.collection.immutable.IndexedSeq[Int]] = Vector(Vector(1), Vector(2, 4), Vector(3, 6, 9))


***
The following two examples show the same but in Java and Python.

```java
import java.util.ArrayList;

class test {
    public static void main(String[] args) {
        ArrayList<Integer> lst = new ArrayList<>();
        lst.add(1);
        lst.add(2);
        lst.add(3);

        ArrayList<ArrayList<Integer>> result = new ArrayList<>();
        ArrayList<Integer> subLst;


        for(int s=0; s < lst.size(); s++) {
            subLst = new ArrayList<>();
            int sValue = lst.get(s);

            for(int t=0; t < s+1; t++) {
                int tValue = sValue * (t + 1);
                subLst.add(tValue);
            }
            result.add(subLst);
        }

        System.out.println(result);
    }
}
```

```python
lst = range(1, 4) # python range is not inclusive

result = []
for i in lst:
    sub_result = [j * i for j in range(1, i+1)]
    result.append(sub_result)

print(result)
```
Since, we want a collections of integers rather than a collection of collections, we start the processing with `flatMap` instead of `map`.

In [6]:
myNums.flatMap(i => (1 to i).map(j => i * j))

res5: scala.collection.immutable.IndexedSeq[Int] = Vector(1, 2, 4, 3, 6, 9)


We can simplify the appearance of the code in the example above by using for-expressions. 

In [7]:
val myNums = (1 to 3).toVector
val result: Vector[Int] = for {
  i <- myNums
  j <- 1 to i
} yield i * j

myNums: Vector[Int] = Vector(1, 2, 3)
result: Vector[Int] = Vector(1, 2, 4, 3, 6, 9)



The rules for using the `for` expressions is that they:
  - must with the keyword `for`
  - must have generators that use `<-`
  - the `yield` keyword will dictate whether or not a new value is returned

`for` expressions are alternative syntax (a.k.a syntactic sugar) for the combined use of `flatMap`, `map`, `withFilter` and `foreach` higher-order functions.

We can also apply **guard conditions** in `for` expressions. Below we modify the earlier example by filtering the original list of `myNums` to odd numbers only.


In [8]:
val myNums = (1 to 3).toVector
val result: Vector[Int] = for {
  i <- myNums if i % 2 == 1
  j <- 1 to i
} yield i * j

myNums: Vector[Int] = Vector(1, 2, 3)
result: Vector[Int] = Vector(1, 3, 6, 9)


In [10]:
val result: Vector[String] = for {
i <- Vector("Ali", "Reza", "Sina")
j <- Vector("Mohammadi", "Karimi", "Safari")
} yield "Hello Mr. " + i + " " + j

result: Vector[String] = Vector(Hello Mr. Ali Mohammadi, Hello Mr. Ali Karimi, Hello Mr. Ali Safari, Hello Mr. Reza Mohammadi, Hello Mr. Reza Karimi, Hello Mr. Reza Safari, Hello Mr. Sina Mohammadi, Hello Mr. Sina Karimi, Hello Mr. Sina Safari)


With this modification we have the results of starting the multiplication with the odd numbers only. You can also modify one of the other earlier examples in Java or Python to achieve the same result.

`for` expression syntax also allows for local definitions. For instance,

In [11]:
case class Time(hours: Int, minutes: Int)

val times = List[Time](Time(10, 10), Time(11, 11), Time(20, 9))

val result: List[String] = for {
  time <- times
  hours = time.hours if hours > 12
} yield (hours - 12) + "pm"

defined class Time
times: List[Time] = List(Time(10,10), Time(11,11), Time(20,9))
result: List[String] = List(8pm)


Note that this is the same as


In [12]:
case class Time(hours: Int, minutes: Int)

val times = List[Time](Time(10, 10), Time(11, 11), Time(20, 9))

val result: List[String] = for {
  time <- times
  if time.hours > 12
} yield (time.hours - 12) + "pm"

defined class Time
times: List[Time] = List(Time(10,10), Time(11,11), Time(20,9))
result: List[String] = List(8pm)


The local definition of `hours` is introduced only to simplify the code below.

All of the code written up to this point has been used to produce values. There are times when we do not want to produce a value. We are executing code to accomplish some sort of side-effect. This could be writing to a database or printing something to the screen or any operation that doesn't return a value but has an external effect. `for` expressions accommodate this situation by allowing us to omit the use of `yield`. 

In [13]:
for (n <- 1 to 3) println(n)

1
2
3


The code above is equivalent to 


In [14]:
(1 to 3).foreach(i => println(i))

1
2
3


To summarize:

  - `for` expressions are a more readable way of expressions consisting of nested `map`, `flatMap`, `filter` and `foreach`.
  - the compiler will translate for expressions that we write into a chain of `map`, `flatMap`, `filter` and `foreach`
  - the syntax allows for filtering and local definitions
  - use `for` expressions whenever you are iterating thought more than one collection-like object or want to chain multiple transformations


# Patterns Matching


Pattern matching is a powerful construct that improves on the familiar `switch`/`case` syntax (which is absent in Scala). It is best to start with a simple example:


In [17]:
val result: Int = 10

result match {
  case 1 => println("found 1")
  case someInteger: Int => println("found some value of type Int")
  // the compiler will issue a warning because this code is acutally impossible
  // to reach. Any value of type Int will match the second condition
  case _ => println("found something that is not 1 or some other integer")
}

found some value of type Int


<console>:33: warning: unreachable code
         case _ => println("found something that is not 1 or some other integer")
                          ^
result: Int = 10


Matching is started with the keyword `match` which is followed by a block of case statements. The left-hand side of the `=>` in the `case` statement supports several varieties of syntax. The right-hand side can contain arbitrary code.

In the first `case` statement, we've checking if the `result` we're trying to find a match for is an integer 1. If it is not, the pattern matching moves on to the next `case` statement. In the following statement, if the value of `result` has a type of `Int`. If that is true, the value of `result` is stored in the name `someInteger`,**this is known as a `type` pattern**. If this pattern match is not successful, ie if `result` is not of type `Int`, the next case is evaluated. This last statement is a catch-all that will successfully match anything. When it matches, the string `"found something that is not 1 or some other integer"` will be printed to the screen.
We can wrap this pattern matching expressions into a function and experiment with it.

In [19]:
def isOneOrInteger(value: Int): Unit =
  value match {
    case 1 => println("found 1")
    case someInteger: Int => println("found some value of type Int")
    case _ => println("found something that is not 1 or some other integer")
  }

isOneOrInteger(1)
isOneOrInteger(22)

found 1
found some value of type Int


<console>:27: warning: unreachable code
           case _ => println("found something that is not 1 or some other integer")
                            ^
isOneOrInteger: (value: Int)Unit



With this definition of the function the compiler will not actually allow us to pass anything other than integers as parameters to the function. In order to get around this, we have to relax the type of `value` to `Any`. Make this change and pass parameters of other types into the function. Observe the output.

Pattern matching is very useful when you want to act on specific values of some case class or a hierarchy of types that represents a set of alternatives (also known as an Algebraic Data Type).

As a reminder, an Algebraic Data Type is a way to encode the fact that a certain type is represented by a finite set of alternatives. For instance, in Scala the Boolean type contains two values, `true` and `false`. We could represent these as distinct types. This is shown below:

In [20]:
sealed trait MyBoolean {
  def value: Boolean
}
case class True(value: Boolean = true) extends MyBoolean
case class False(value: Boolean = false) extends MyBoolean

defined trait MyBoolean
defined class True
defined class False


This the typical representation of an ADT in Scala. Closing the `trait` with `sealed` means that all of the definitions that are inherit from this trait  are defined in the same Scala file where this definition is located.


As another example, we could model the behavior of a light switch with an ADT. This is shown below:

In [21]:
sealed trait LightSwitch
case object On extends LightSwitch
case object Off extends LightSwitch

defined trait LightSwitch
defined object On
defined object Off


Since we've notified the compiler, by using `sealed`, that all of the subtypes of `LightSwitch` are in this file, it (the compiler) will verify that we have covered all of the cases when we pattern match on a value of type `LightSwitch`. Assign a value to `light` and observe the behavior.


In [22]:
sealed trait LightSwitch
case object On extends LightSwitch
case object Off extends LightSwitch

val light: LightSwitch = Off

def status(value: LightSwitch): Unit = 
  value match {
    case On => println("the switch is on")
    case Off => println("this switch is off")
  }

status(light)

this switch is off


defined trait LightSwitch
defined object On
defined object Off
light: LightSwitch = Off
status: (value: LightSwitch)Unit



We can write something similar to the above for our custom `Boolean` type. These values are little bit more complex in that they actually carry some values. However, using pattern matching we can de-compose or `unapply` case class into its components. Set value of `b` to one of the possible alternatives and observe the difference.

In [24]:
sealed trait MyBoolean {
  def value: Boolean
}

case class True(override val value: Boolean = true) extends MyBoolean
case class False(override val value: Boolean = false) extends MyBoolean


val b: MyBoolean = False()

def isItActuallyTrue(value: MyBoolean): Unit =
  value match {
    case True(true) => println("it is actually true!")
    case True(false) => println("it is pretending to be true!")
    case False(true) => println("it is pretending to be false, but is actually true!")
    case False(false) => println("it is really false!")
  }

defined trait MyBoolean
defined class True
defined class False
b: MyBoolean = False(false)
isItActuallyTrue: (value: MyBoolean)Unit


# Options

`Option` represents a presence or an absence of a value. Consider division. We could implement division and handle division by 0 by throwing an exception like so:


In [26]:
def divE(numerator: Int, denominator: Int): Int =
  if (denominator == 0) throw new Exception("Denominator is 0") else numerator / denominator

divE(10, 2)

divE: (numerator: Int, denominator: Int)Int
res16: Int = 5


In [27]:
divE(2, 0)

java.lang.Exception:  Denominator is 0

However, this is far from idiomatic Scala. In the above we interrupting the flow of the program with an exception that we can easily handle ourselves. Furthermore, there is nothing about the function definition that communicates to us that this function may not produce a value in some cases. The following is a better definition of the same function:

In [28]:
def divO(numerator: Int, denominator: Int): Option[Int] =
  if (denominator == 0) None else Some(numerator / denominator)

divO(10, 2)
divO(2, 0)

divO: (numerator: Int, denominator: Int)Option[Int]
res18: Option[Int] = None


wow!

Pattern matching makes processing values of type `Option` straightforward.


In [30]:
def stringLegnth(value: Option[String]): Int =
  value match {
    case Some(str) => str.length
    case None => 0
  }

stringLegnth: (value: Option[String])Int


This pattern, where we want to extract the value from an `Option` or provide a default value when the it is None occurs frequently. It is more concisely represented by the function `getOrElse`. Thus, we can reimplement the functionality of `stringLegnth` as: 

In [40]:
def stringLegnth(value: Option[String]): Int =
  value.getOrElse("").length

stringLegnth: (value: Option[String])Int


We provide the default value of `""` (empty string) for the case when `value` is `None`. 

There are a number of higher order functions we can use with `Option`. For example, another way to implement `stringLegnth` to combine `map` and `getOrElse`.

In [41]:
def stringLegnth(value: Option[String]): Int =
  value.map(str => str.length).getOrElse(0)

stringLegnth: (value: Option[String])Int


If `value` is a `Some` we will compute the length of the string that it contains. If it is `None`, `map` will not do anything. Subsequently, `getOrElse` will be called. Note, that `getOrElse` is now being called on an `Option[Int]` value, because by using `map` we have changed the type of the value inside the `Option` from `String` to `Int`. As always, `getOrElse` will either return the contents of the `Option` or the default value of 0.

We can sequence processing of options with `flatMap`. For example,

In [42]:
def fullName(firstName: Option[String], lastName: Option[String]): Option[String] =
  firstName.flatMap(fn => lastName.map(ln => s"$fn $ln"))

val bothOption: Option[String] = fullName(Option("John"), Option("Doe"))
val oneOption: Option[String] = fullName(Option("Prince"), None)

fullName: (firstName: Option[String], lastName: Option[String])Option[String]
bothOption: Option[String] = Some(John Doe)
oneOption: Option[String] = None


As you may remember the section on `for` expressions, nested combinations of `map` and `flatMap` can be re-written using `for` expressions. Thus, the `fullName` function above can be written as:

In [44]:
def fullName(firstName: Option[String], lastName: Option[String]): Option[String] =
  for {
    fn <- firstName
    ln <- lastName
  } yield s"$fn $ln"

println(fullName(Option("John"), Option("Doe")))
println(fullName(Option("Prince"), None))

Some(John Doe)
None


fullName: (firstName: Option[String], lastName: Option[String])Option[String]


# Handling Failure

There are a number of mechanisms that we can use in Scala to handle failures. In particular, we have already seen that instead of throwing exceptions we can avoid them by using `Option`. The Scala standard library provides another abstraction to help us with code where we have to handle an exception rather than avoiding it, `Try`.

`Try` is actually an ADT that consists of `Success` and `Failure`. `Success` captures the result of the successful computation. `Failure` captures the exception that occurred during the computation. This is shown below.


In [48]:
import scala.util.{Try, Success, Failure}

def divT(numerator: Int, denominator: Int): Try[Int] =
  Try(numerator / denominator)

val result: Try[Int] = divT(10, 0)

result match {
  case Success(v) => println("got the result of: " + v)
  case Failure(e) => println("got an exception of: " + e)
}

got an exception of: java.lang.ArithmeticException: / by zero


import scala.util.{Try, Success, Failure}
divT: (numerator: Int, denominator: Int)scala.util.Try[Int]
result: scala.util.Try[Int] = Failure(java.lang.ArithmeticException: / by zero)


As can be seen above, we can use pattern matching to process a value of type `Try`. Similar to `Option`, `Try` implements a number of higher order functions. Some of these functions are shown below.

In [49]:
import scala.util.{Try, Success, Failure}

def divT(numerator: Int, denominator: Int): Try[Int] =
  Try(numerator / denominator)

val result: Try[Int] = divT(10, 0)

// getOrElse on Try works the same was as on Option. note that the value has to be the same type as the success case
val getOrElseValue: Int = result.getOrElse(0)

// similar to Option, if `result` is a `Success` add 10 to it. Otherwise, do nothing
val mapValue: Try[Int] = result.map(success => success + 10)

// just like with `Option` we can sequence try computations with `flatMap`
val flatMapValue: Try[Int] = result.flatMap(success => divT(success, 10))

// transform takes two functions as parameters, one for each type of result
val transformValue: Try[Int] = result.transform(s => Success(s + 10), ex => Success(0))

import scala.util.{Try, Success, Failure}
divT: (numerator: Int, denominator: Int)scala.util.Try[Int]
result: scala.util.Try[Int] = Failure(java.lang.ArithmeticException: / by zero)
getOrElseValue: Int = 0
mapValue: scala.util.Try[Int] = Failure(java.lang.ArithmeticException: / by zero)
flatMapValue: scala.util.Try[Int] = Failure(java.lang.ArithmeticException: / by zero)
transformValue: scala.util.Try[Int] = Success(0)


Since we have `map` and `flatMap` implemented on `Try`, we can use `for` expressions with `Try`.


In [51]:
import scala.util.{Try, Success, Failure}

def divT(numerator: Int, denominator: Int): Try[Int] =
  Try(numerator / denominator)

val result: Try[Int] = for {
  r1 <- divT(10, 10)
  r2 <- divT(1100, 34)
} yield r1 * r2

import scala.util.{Try, Success, Failure}
divT: (numerator: Int, denominator: Int)scala.util.Try[Int]
result: scala.util.Try[Int] = Success(32)


The above can also be represented as:

In [52]:
import scala.util.{Try, Success, Failure}

def divT(numerator: Int, denominator: Int): Try[Int] =
  Try(numerator / denominator)

val resultForExpression: Try[Int] = for {
  r1 <- divT(10, 10)
  r2 <- divT(1100, 34)
} yield r1 * r2

val resultFlatMap = divT(10, 10).flatMap(r1 => divT(1100, 34).map(r2 => r1 * r2))

import scala.util.{Try, Success, Failure}
divT: (numerator: Int, denominator: Int)scala.util.Try[Int]
resultForExpression: scala.util.Try[Int] = Success(32)
resultFlatMap: scala.util.Try[Int] = Success(32)


# Futures

By relying on immutability Scala makes multi-threaded programming easier. In addition to this, the Scala standard library provides an additional abstract to represent computations that may happen at a later time or on another thread. This abstraction is represented by the `Future` type.

Prior to being able to create values of type `Future` we must specify on what thread pool the execution will be performed. The framework you're working with (like Play or Akka) will typically provide a way for you to use its own thread pool. In Scala, Java thread pools as wrapped into values of type `ExecutionContext`. Creating an `ExecutionContext` is shown below:

In [53]:
import scala.concurrent.ExecutionContext
import java.util.concurrent.ForkJoinPool

val ec: ExecutionContext = ExecutionContext.fromExecutor(new ForkJoinPool())

import scala.concurrent.ExecutionContext
import java.util.concurrent.ForkJoinPool
ec: scala.concurrent.ExecutionContext = scala.concurrent.impl.ExecutionContextImpl@5d861b9a


with this value in scope we can start creating values of type `Future`.


In [54]:
import scala.concurrent.{ExecutionContext, Future}
import java.util.concurrent.ForkJoinPool

val ec: ExecutionContext = ExecutionContext.fromExecutor(new ForkJoinPool())

// use `.successful` if simply wrapping a static value into a `Future`. This doesn't create threads
val f1: Future[Int] = Future.successful(1)

def longComputation(duration: Int): Int = {
  // pretend as if time consuming work is being done
  import java.lang.Thread.sleep
  sleep(duration)
  duration
}

// this will be executed in a thread on the ExecutionContext
// we are passing the ExecutionContext value explicitly
val f2: Future[Int] = Future(longComputation(100))(ec)

import scala.concurrent.{ExecutionContext, Future}
import java.util.concurrent.ForkJoinPool
ec: scala.concurrent.ExecutionContext = scala.concurrent.impl.ExecutionContextImpl@76b57d6
f1: scala.concurrent.Future[Int] = Future(Success(1))
longComputation: (duration: Int)Int
f2: scala.concurrent.Future[Int] = Future(<not completed>)


Passing the `ExecutionContext` value every time we use a `Future` is error-prone and hards readability. `Future` instances can take this parameter as an implicit parameter. Thus, we can write,


In [55]:
import scala.concurrent.{ExecutionContext, Future}
import java.util.concurrent.ForkJoinPool

implicit val ec: ExecutionContext = ExecutionContext.fromExecutor(new ForkJoinPool())

def longComputation(duration: Int): Future[Int] = {
  // pretend as if time consuming work is being done
  import java.lang.Thread.sleep
  Future {
    sleep(duration)
    duration
  }
}

val f3: Future[Int] = longComputation(100)

import scala.concurrent.{ExecutionContext, Future}
import java.util.concurrent.ForkJoinPool
ec: scala.concurrent.ExecutionContext = scala.concurrent.impl.ExecutionContextImpl@5afdf7d0
longComputation: (duration: Int)scala.concurrent.Future[Int]
f3: scala.concurrent.Future[Int] = Future(<not completed>)



Note that when we run code like `Futuer(longComputation(100))`, the result that is returned an `Future` that has not yet been completed. Eventually, the value will be computed and available. However, this doesn't prevent as from writing code that will be executed after the value as been fulfilled. 

For example, we can pattern match on `Future`.

In [56]:
import scala.concurrent.{ExecutionContext, Future}
import scala.util.{Success, Failure}
import java.util.concurrent.ForkJoinPool

implicit val ec: ExecutionContext = ExecutionContext.fromExecutor(new ForkJoinPool())

def longComputation(duration: Int): Future[Int] = {
  // pretend as if time consuming work is being done
  import java.lang.Thread.sleep
  Future {
    sleep(duration)
    duration
  }
}

val f4: Future[String] = Future {
  import java.lang.Thread.sleep
  sleep(100)
  "hello " + "world!"
}

f4.onComplete {
  case Success(value) => println("!" + value)
  case Failure(ex) => println("something went wrong. we took too long to add two strings!")
}

import scala.concurrent.{ExecutionContext, Future}
import scala.util.{Success, Failure}
import java.util.concurrent.ForkJoinPool
ec: scala.concurrent.ExecutionContext = scala.concurrent.impl.ExecutionContextImpl@2ef92091
longComputation: (duration: Int)scala.concurrent.Future[Int]
f4: scala.concurrent.Future[String] = Future(<not completed>)


!hello world!


We can use familiar higher order functions and `for` expressions.


In [57]:
import scala.concurrent.{ExecutionContext, Future}
import scala.util.{Success, Failure}
import java.util.concurrent.ForkJoinPool

implicit val ec: ExecutionContext = ExecutionContext.fromExecutor(new ForkJoinPool())

def longComputation(duration: Int): Future[Int] = {
  // pretend as if time consuming work is being done
  import java.lang.Thread.sleep
  Future {
    sleep(duration)
    duration
  }
}

val f4: Future[String] = Future {
  import java.lang.Thread.sleep
  sleep(100)
  "hello " + "world!"
}

// use the result if the Future was successful, otherwise, do nothing.
val f6: Future[Int] = longComputation(100).map(successfulResult => successfulResult + 100)
val f7: Future[String] = longComputation(100).flatMap(r1 => longComputation(100).map(r2 => r1.toString + r2.toString))

val f8: Future[String] = for {
  r1 <- longComputation(100)
  r2 <- longComputation(100)
} yield (r1.toString + r2.toString)

import scala.concurrent.{ExecutionContext, Future}
import scala.util.{Success, Failure}
import java.util.concurrent.ForkJoinPool
ec: scala.concurrent.ExecutionContext = scala.concurrent.impl.ExecutionContextImpl@44b699e1
longComputation: (duration: Int)scala.concurrent.Future[Int]
f4: scala.concurrent.Future[String] = Future(<not completed>)
f6: scala.concurrent.Future[Int] = Future(<not completed>)
f7: scala.concurrent.Future[String] = Future(<not completed>)
f8: scala.concurrent.Future[String] = Future(<not completed>)


There are many additional methods on `Future` that can be helpful. For example, `recover` allows us to specify how to handle specific types of exceptions. This is similar to `onComplete` but we do not have to specify the success case.

In [58]:
import scala.concurrent.{ExecutionContext, Future}
import scala.util.{Success, Failure}
import java.util.concurrent.ForkJoinPool

implicit val ec: ExecutionContext = ExecutionContext.fromExecutor(new ForkJoinPool())

def longComputation(duration: Int): Future[Int] = Future {
  // pretend as if time consuming work is being done
  import java.lang.Thread.sleep
  sleep(duration)
  throw new IllegalStateException
}

longComputation(100).recover {
  case _: IllegalStateException => 0
}

import scala.concurrent.{ExecutionContext, Future}
import scala.util.{Success, Failure}
import java.util.concurrent.ForkJoinPool
ec: scala.concurrent.ExecutionContext = scala.concurrent.impl.ExecutionContextImpl@16af6a2b
longComputation: (duration: Int)scala.concurrent.Future[Int]
res34: scala.concurrent.Future[Int] = Future(<not completed>)


There are many other functions and details to the workings of `Future`. The Scala documentation provides a great overview of most of the features and additional functionality. You can find those details here: [https://docs.scala-lang.org/overviews/core/futures.html](https://docs.scala-lang.org/overviews/core/futures.html)

***
### part of Scala 101: cognitiveclass.ai lab